Helper module to handle linear inequalities.

In [2]:
%run constants.ipynb
from sympy import And, Not, simplify, Add

In [4]:
EPS = 1e-10
def nonstricten(*ineqs):
    nonstrict = []
    for i in ineqs:
        if isinstance(i, Gt):
            # a>b => a >= b+EPS
            nonstrict.append(Ge(i.lhs, i.rhs + EPS))
        elif isinstance(i, Lt):
            # a<b => a <= b-EPS
            nonstrict.append(Le(i.lhs, i.rhs - EPS))
        else:
            nonstrict.append(i)  # already non-strict
    return nonstrict

In [5]:
from sympy.solvers.simplex import lpmax, InfeasibleLPError
from sympy.core import *

def solve_lp(*constraints, Ax=0):
    ''' Returns max LP solution for Ax <= b, under the given `constraints`. '''
    return lpmax(Ax, nonstricten(*constraints))

def feasible(*ineqs, r=V):
    ''' Returns true if the And of the linear inequalities `ineqs` are feasible under constraints `r`, 
        false otherwise. 
    '''
    try:
        result = solve_lp(*r, *ineqs, Ax=0)
    except InfeasibleLPError:
        return False
    return result

def redundant(ineq, r=V):
    ''' Returns true if the linear inequality `ineq` is redundant under constraints `r`, 
        false otherwise. 
    '''
    # if (Assumptions => Ineq) = (not A or I) is valid, then (A and not I) is unsatisfiable
    return feasible(Not(ineq), r=r) == False

def non_redundant(*ineqs, r=V):
    ''' Returns the non-redundant inequalities from `ineqs` under constraints `r`.
    '''
    return [i for i in ineqs if not redundant(i, r=r)]

def equiv(rs, ss):
    ''' Returns true if e1 = e2, false otherwise.
    '''
    rs = set([simplify(r) for r in rs])
    ss = set([simplify(s) for s in ss])
    return all([r.equals(s) for r,s in zip(rs, ss)])

def opposite(rs, ss):
    return len(rs) == 1 and len(ss) == 1 and simplify(Not(rs[0])).equals(simplify(ss[0]))

In [6]:
def reduce(inds, r=V):
    ''' 
    Reduces the 2D list of `inds` under restriction `r`.
    i.e. Inferrable indicators from `r` are converted to 0s or 1s.
    Returns a 2-tuple of constant value (evaluated indicators), remaining indicators.
    '''
    reduced = []
    const = 0
    for i in inds:
        if not feasible(*i, r=r):
            # not feasible
            continue
        else:
            rem = non_redundant(*i, r=r)
            if not rem:
                const += 1
            else:
                reduced.append(rem)
                
    return const, reduced

In [8]:
def sub(ind, vals):
    '''
    Evaluates the (compound) indicator constraint `ind`, 
    using substitution of values from `vals` (a variable : value dictionary).
    '''
    return 1 if And(*ind).subs(vals) else 0